In [1]:
#MASTER EQUATION SOLVER
#FIRST lets import all the necessary libraries that are needed for our desired output
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import qutip as qt
from ipywidgets import interact, FloatSlider, Dropdown

# for interacting plotting,we beed this"ipympl library"
%matplotlib ipympl


sigma_x = qt.sigmax()
sigma_z = qt.sigmaz()

#Its a Global variable whose role is to retain the animation object
ani = None

def qubit_dynamics(qubit_type, delta, beta, t1, tf):
    global ani

    if qubit_type == 'Single spin':#it makes a drop down,when only spin down will be given as option
        # Here,we Define Hamiltonians for system and environment
        H_system = delta * sigma_z
        H_environment = beta * sigma_x
        H = H_system + H_environment

       
        psi0 = qt.basis(2, 0)

       
        tlist = np.linspace(t1, tf, 100)

        
        result = qt.mesolve(H, psi0, tlist, [], [sigma_z])
        expect_z = result.expect[0]
        #plotting begins here:both static and animated plots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

        # To make the static plot:The needed axes and labels
        ax1.plot(tlist, expect_z, label=r'$\langle \sigma_z \rangle$', color='orange')
        ax1.set_xlabel('Time(t)')
        ax1.set_ylabel('Expectation Value of $\sigma_z$')
        ax1.set_title('Task 2: Qubit Dynamics 0.1')
        ax1.legend()
        ax1.grid()

        # To make the animated plot:The needed axes and labels
        ax2.set_xlim(t1, tf)
        ax2.set_ylim(min(expect_z)-0.1, max(expect_z)+0.1)
        ax2.set_xlabel('Time(t)')
        ax2.set_ylabel('Expectation Value of $\sigma_z$')
        ax2.set_title('Optional Task: Qubit Dynamics(animated)')
        line, = ax2.plot([], [], lw=2, color='purple', label=r'$\langle \sigma_z \rangle$')
        ax2.legend()
        ax2.grid()

        #To animate,we define here two functions as follows:
        def init():
            line.set_data([], [])
            return (line,)

        def update(frame):
            line.set_data(tlist[:frame], expect_z[:frame])
            return (line,)

        #Animation:
        ani = animation.FuncAnimation(fig, update, frames=len(tlist),
                                      init_func=init, blit=True, interval=50)
        
        plt.tight_layout()
        plt.show()
        return ani

# Interactive user inputs using "interact funcion" which we define here:
interact(
    qubit_dynamics,
    qubit_type=Dropdown(options=['Single spin'], value='Single spin', description='Qubit type:'),
    delta=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Delta:'),
    beta=FloatSlider(value=0.5, min=0.1, max=5.0, step=0.1, description='Beta:'),
    t1=FloatSlider(value=0.0, min=0.0, max=10.0, step=0.1, description='t1:'),
    tf=FloatSlider(value=10.0, min=1.0, max=20.0, step=0.1, description='tf:')
); 

interactive(children=(Dropdown(description='Qubit type:', options=('Single spin',), value='Single spin'), Floa…